In [ ]:
    import warnings
    warnings.filterwarnings("ignore")
    from sklearn import datasets
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    import matplotlib.pyplot as plt
    import pandas as pd
    import seaborn as sns
    import numpy as np
    import tqdm
    plt.style.use('ggplot')
    import lightgbm as ltb

In [ ]:
import random

# Set the seed to 42 (you can use any integer value)
random.seed(42)


In [ ]:
    import warnings
    warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv("/kaggle/input/valid-dataset-adobe/valid_paths_data.csv")


In [ ]:
text_embedding_path="/kaggle/input/clipx32-batch-mpnetx8-batch/MPNET_Embeds.npy"

In [ ]:
image_embedding_path="/kaggle/input/final-image-embeddings-efnet/ENet_Embeds_final.npy"

In [ ]:
def load_data(text_embedding_path, image_embedding_path, data):
    text_embeddings = np.load(text_embedding_path)
    image_embeddings = np.load(image_embedding_path)

    
    features = np.array(data[['hour','month', 'Year', 'day of week']])
    
    features = np.concatenate([features,text_embeddings,image_embeddings], axis=1)

    target = np.array(data["likes"])

    return features, target

In [ ]:
X,y=load_data(text_embedding_path, image_embedding_path, data)

In [ ]:
y=np.log(y+1)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



# Specify the fraction of the dataset to keep (e.g., 0.1 for 10%)
subsample_fraction = 0.6

# Create bins or quantiles of the target variable
bins = np.linspace(y.min(), y.max(), num=5)  # Adjust the number of bins as needed
y_binned = pd.cut(y, bins=bins, labels=False, include_lowest=True)

# Perform stratified subsampling based on bins
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1 - subsample_fraction, random_state=42, stratify=y_binned
)

In [ ]:
X_train.shape

In [ ]:
import pandas as pd
import lightgbm as lgb
 
# Similarly LGBMRegressor can also be imported for a regression model.
from lightgbm import LGBMRegressor

In [ ]:
my_model = LGBMRegressor(n_estimators=10000,num_leaves=64,lambda_l2=0.1,lambda_l1=0.1,bagging_fraction = 0.8,bagging_freq = 1,min_data_in_leaf=300, learning_rate=0.1, n_jobs=-1,objective='regression', metric='neg_mean_squared_error',verbose=1)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=100, 
             eval_set=(X_test, y_test), 
             eval_metric="mse",verbose=True)

In [ ]:
from sklearn.metrics import mean_squared_error

predictions = my_model.predict(X_train)
print("Mean squared Error: " + str(mean_squared_error((np.exp(predictions)-1), (np.exp(y_train)-1))))

In [ ]:
from sklearn.metrics import mean_squared_error

predictions = my_model.predict(X_train)
print("Mean squared Error: " + str(mean_squared_error(predictions, y_train)))

In [ ]:
from sklearn.metrics import mean_squared_error

predictions = my_model.predict(X_test)
print("Mean squared Error: " + str(mean_squared_error((np.exp(predictions)-1), (np.exp(y_test)-1))))

In [ ]:
import pickle

In [ ]:
with open('lgbm.pkl', 'wb') as f:
    pickle.dump(my_model, f)

In [ ]:
with open('lgbm.pkl', 'rb') as f:
    model = pickle.load(f)